In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Number of symbols
N = 100000

# Generate input symbols (BPSK modulation)

ip = np.random.randn(N) > 0.5

# Convert input symbols to BPSK symbols (+1 or -1)
s = 2 * ip - 1

# Create an array for storing Bit Error Rate (BER) values at different Eb/No levels
eb_no_db = np.arange(0, 35)
total_error = np.zeros(len(eb_no_db))

# Loop through different Eb/No levels
for ii in range(len(eb_no_db)):

    nErr = 0  # Initialize the error count for this Eb/No level

    # Loop through the symbols
    for jj in range(N // 2):

        # Generate Rayleigh fading coefficients for two transmit antennas
        h0 = (1 / np.sqrt(2)) * (np.random.randn(N) + 1j * np.random.randn(N))
        h1 = (1 / np.sqrt(2)) * (np.random.randn(N) + 1j * np.random.randn(N))

        # Generate complex Gaussian noise for two receive antennas
        n0 = (1 / np.sqrt(2)) * (np.random.randn(N) + 1j * np.random.randn(N))
        n1 = (1 / np.sqrt(2)) * (np.random.randn(N) + 1j * np.random.randn(N))

        # Simulate received signals with Rayleigh fading and noise
        r0 = h0 * s[2 * jj] + h1 * s[2 * jj + 1] + n0 * 10 ** (-eb_no_db[ii] / 20)
        r1 = -(h0 * np.conj(s[2 * jj + 1])) + h1 * np.conj(s[2 * jj]) + n1 * 10 ** (-eb_no_db[ii] / 20)

        # Perform Alamouti Space-Time Block Coding (STBC) decoding
        s0_tilde = np.conj(h0) * r0 + h1 * np.conj(r1)
        s1_tilde = np.conj(h1) * r0 - h0 * np.conj(r1)

        # Equalization by dividing by the sum of absolute fading coefficients
        s0_tilde = s0_tilde / (np.abs(h0) + np.abs(h1))
        s1_tilde = s1_tilde / (np.abs(h0) + np.abs(h1))

        # BPSK demodulation
        received_bits_s0 = np.real(s0_tilde) >= 0
        received_bits_s1 = np.real(s1_tilde) >= 0

        # Compare received bits with original input bits and count errors
        if not np.all(received_bits_s0 == ip[2 * jj]):
            nErr += 1
        if not np.all(received_bits_s1 == ip[2 * jj + 1]):
            nErr += 1

    # Calculate Bit Error Rate (BER) for this Eb/No level and store it in the array
    total_error[ii] = nErr / (2 * N)  # Divide by (2 * N) because there are 2 bits per symbol

    # Print the current BER values for monitoring (optional)
    # print(total_error)

# Plot the Bit Error Rate (BER) vs. Eb/No (signal-to-noise ratio) in dB
plt.figure()
plt.semilogy(eb_no_db, total_error, 'ks-', linewidth=2, label='nTx=2 nRx=1 (sim)')
plt.axis([0, 35, 1e-5, 0.5])
plt.legend()
plt.xlabel('Eb/No, dB')
plt.ylabel('Bit Error Rate')
plt.title('BER for BPSK modulation with Alamouti STBC in Rayleigh channel')
plt.grid(True)
plt.show()
